# Linear Assignment

This section replicates the example from chapter 4 of the book
[Opt Art by Robert Bosch](https://www.amazon.com/Opt-Art-Mathematical-Optimization-Visual/dp/0691164061).

# Problem description

In this problem we need to assign professors to courses in a one-on-one fashion.

- There are 5 professors.
- There are 5 courses.
- Professors will complain when they need to give a course, but the amount of complaining differs. This is the cost table.
- Find an assignment with minimal cost.

This class of problems is known as _linear assignment_.

# Problem formulation

We will use [PuLP](https://pypi.org/project/PuLP/) (a linear programming modeler written in python).

In [1]:
import pulp

We have five professors and five courses, leading to 25 combinations.

In [2]:
profs = [ 'profA', 'profB', 'profC', 'profD', 'profE' ]
courses = [ 'course1', 'course2', 'course3', 'course4', 'course5' ]
combos = [ (p,c) for p in profs for c in courses ]
combos

[('profA', 'course1'),
 ('profA', 'course2'),
 ('profA', 'course3'),
 ('profA', 'course4'),
 ('profA', 'course5'),
 ('profB', 'course1'),
 ('profB', 'course2'),
 ('profB', 'course3'),
 ('profB', 'course4'),
 ('profB', 'course5'),
 ('profC', 'course1'),
 ('profC', 'course2'),
 ('profC', 'course3'),
 ('profC', 'course4'),
 ('profC', 'course5'),
 ('profD', 'course1'),
 ('profD', 'course2'),
 ('profD', 'course3'),
 ('profD', 'course4'),
 ('profD', 'course5'),
 ('profE', 'course1'),
 ('profE', 'course2'),
 ('profE', 'course3'),
 ('profE', 'course4'),
 ('profE', 'course5')]

We also have a table of costs.

In [3]:
costs_matrix = [ 
    2.5, 3.5, 4.0, 4.0, 3.5,
    2.5, 1.5, 2.5, 1.5, 4.0,
    4.0, 3.0, 3.5, 2.0, 3.5,
    3.5, 4.0, 3.5, 2.5, 4.0,
    4.0, 3.5, 3.5, 4.0, 1.5 
]

It should be interpreted as follows.

In [4]:
costs = dict( zip(combos,costs_matrix) )
costs

{('profA', 'course1'): 2.5,
 ('profA', 'course2'): 3.5,
 ('profA', 'course3'): 4.0,
 ('profA', 'course4'): 4.0,
 ('profA', 'course5'): 3.5,
 ('profB', 'course1'): 2.5,
 ('profB', 'course2'): 1.5,
 ('profB', 'course3'): 2.5,
 ('profB', 'course4'): 1.5,
 ('profB', 'course5'): 4.0,
 ('profC', 'course1'): 4.0,
 ('profC', 'course2'): 3.0,
 ('profC', 'course3'): 3.5,
 ('profC', 'course4'): 2.0,
 ('profC', 'course5'): 3.5,
 ('profD', 'course1'): 3.5,
 ('profD', 'course2'): 4.0,
 ('profD', 'course3'): 3.5,
 ('profD', 'course4'): 2.5,
 ('profD', 'course5'): 4.0,
 ('profE', 'course1'): 4.0,
 ('profE', 'course2'): 3.5,
 ('profE', 'course3'): 3.5,
 ('profE', 'course4'): 4.0,
 ('profE', 'course5'): 1.5}

We have now all ingredients to formalize the problem.

First, we create the problem _object_. We want to minimize cost.

In [5]:
prob = pulp.LpProblem("teaching", pulp.LpMinimize)

Next, we create the variables. They indicate which professor is assigned to which course. All variables must be 0 or 1. 
Quoting Robert Bosch: 

> "Researchers have proved that in case of the linear assignment problem, every vertex of the continuous relaxation feasible region is integer valued."

In [6]:
assign = pulp.LpVariable.dicts( "assign", combos, lowBound=0, upBound=1 ) # We can also use: cat=pulp.LpBinary

Thirdly, the objective function is the cost associated with the assignment.

In [7]:
prob += pulp.lpSum( [assign[(p,c)]*costs[(p,c)] for p in profs for c in courses] ), "lowest cost"

Each professor can only be assigned to one course.

In [8]:
for p in profs:
    prob += pulp.lpSum( [assign[(p,c)] for c in courses] ) == 1 , f"{p} once"

Finally, each course can only be assigned to one professor.

In [9]:
for c in courses:
    prob += pulp.lpSum( [assign[(p,c)] for p in profs] ) == 1 , f"{c} once"

And we are done. Let's save and view the problem definition.

In [10]:
prob.writeLP("LinAssign.lp")
!type LinAssign.lp

\* teaching *\
Minimize
lowest_cost: 2.5 assign_('profA',_'course1') + 3.5 assign_('profA',_'course2')
 + 4 assign_('profA',_'course3') + 4 assign_('profA',_'course4')
 + 3.5 assign_('profA',_'course5') + 2.5 assign_('profB',_'course1')
 + 1.5 assign_('profB',_'course2') + 2.5 assign_('profB',_'course3')
 + 1.5 assign_('profB',_'course4') + 4 assign_('profB',_'course5')
 + 4 assign_('profC',_'course1') + 3 assign_('profC',_'course2')
 + 3.5 assign_('profC',_'course3') + 2 assign_('profC',_'course4')
 + 3.5 assign_('profC',_'course5') + 3.5 assign_('profD',_'course1')
 + 4 assign_('profD',_'course2') + 3.5 assign_('profD',_'course3')
 + 2.5 assign_('profD',_'course4') + 4 assign_('profD',_'course5')
 + 4 assign_('profE',_'course1') + 3.5 assign_('profE',_'course2')
 + 3.5 assign_('profE',_'course3') + 4 assign_('profE',_'course4')
 + 1.5 assign_('profE',_'course5')
Subject To
course1_once: assign_('profA',_'course1') + assign_('profB',_'course1')
 + assign_('profC',_'course1') + assign_

## Solving

At this moment the `prop` problem is not yet solved.

In [11]:
pulp.LpStatus[prob.status]

'Not Solved'

We solve it, and check the status for success.

In [12]:
prob.solve()
pulp.LpStatus[prob.status]

'Optimal'

The cost of the optimal assignment is now also computable. Note that `pulp.value(E)` computes the value of expression `E`. The `prob.objective` is indeed an expression computing the cost.

In [13]:
print(prob.objective)

2.5*assign_('profA',_'course1') + 3.5*assign_('profA',_'course2') + 4.0*assign_('profA',_'course3') + 4.0*assign_('profA',_'course4') + 3.5*assign_('profA',_'course5') + 2.5*assign_('profB',_'course1') + 1.5*assign_('profB',_'course2') + 2.5*assign_('profB',_'course3') + 1.5*assign_('profB',_'course4') + 4.0*assign_('profB',_'course5') + 4.0*assign_('profC',_'course1') + 3.0*assign_('profC',_'course2') + 3.5*assign_('profC',_'course3') + 2.0*assign_('profC',_'course4') + 3.5*assign_('profC',_'course5') + 3.5*assign_('profD',_'course1') + 4.0*assign_('profD',_'course2') + 3.5*assign_('profD',_'course3') + 2.5*assign_('profD',_'course4') + 4.0*assign_('profD',_'course5') + 4.0*assign_('profE',_'course1') + 3.5*assign_('profE',_'course2') + 3.5*assign_('profE',_'course3') + 4.0*assign_('profE',_'course4') + 1.5*assign_('profE',_'course5')


In [14]:
pulp.value(prob.objective)

11.0

I wanted to print the assignment, but I was a bit puzzled by the data structures. The `assign` object is a dictionary. Its _keys_ are the ones passed when creating `assign`, in our case that is `combos`. In `combos` we have two-tuples of strings like `('profA', 'course1')`. Indeed those are the keys of `assign`:

In [15]:
for k in assign.keys() :
    print(k)

('profA', 'course1')
('profA', 'course2')
('profA', 'course3')
('profA', 'course4')
('profA', 'course5')
('profB', 'course1')
('profB', 'course2')
('profB', 'course3')
('profB', 'course4')
('profB', 'course5')
('profC', 'course1')
('profC', 'course2')
('profC', 'course3')
('profC', 'course4')
('profC', 'course5')
('profD', 'course1')
('profD', 'course2')
('profD', 'course3')
('profD', 'course4')
('profD', 'course5')
('profE', 'course1')
('profE', 'course2')
('profE', 'course3')
('profE', 'course4')
('profE', 'course5')


One of the keys of `assign` is `('profA', 'course1')` and we can extract the parts with `key[0]` (giving `'profA'`) and `key[1]` (giving `'course1'`). 

When we have a key we can use that as in index into `assign`, and we get an LP variable, more precisely an object of type `pulp.pulp.LpVariable`. 

In [16]:
k = ('profA', 'course1')
v = assign[k]
type(v)

pulp.pulp.LpVariable

An LP variable is an object from the pulp package. It has 83 methods. 

In [17]:
len( dir( v ) )

83

An LP variable can be printed, which is coded as printing the variable's _name_. In our case the names are quite unreadable because they are based on the keys we passed. Here is an example of the name of a variable.

In [18]:
print(v)

assign_('profA',_'course1')


We saw names like this pop up in the file `LinAssign.lp`. 

Out of the 83 methods an LP variable has, two seem important method, `name()` and `value()`. 
The former returns the above mentioned name.

In [19]:
v.name

"assign_('profA',_'course1')"

The latter, `value()`, returns the optimal value, at least, _after_ solving.

In [20]:
v.value()

1.0

With this knowledge, we can print which professor teaches which course and at what cost.

In [21]:
cost = 0
for key,var in assign.items():
    if var.value() :
        print( f"{key[0]} - {key[1]} - {costs[key]}" )
        cost += costs[key]
print(cost)

profA - course1 - 2.5
profB - course2 - 1.5
profC - course4 - 2.0
profD - course3 - 3.5
profE - course5 - 1.5
11.0


Indeed, the cost of this is 11 (2.5+1.5+2.0+3.5+1.5).

(end)